In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = "/home/jackson/research/data/tortuosity_study/read_csvs/score_tables/r4_scores.csv"

In [ ]:
df = pd.read_csv(path)

In [ ]:
df

In [ ]:
arr = df.to_numpy()

In [ ]:
def majority_consensus(scores: list[int], target_score: int, consensus_thresh: int) -> True:
    return scores.count(target_score) >= consensus_thresh


Calculate the number of images in the final read with a 2/3 consensus

In [ ]:
consensus_slides = []
threshhold = 2
for index, scores in enumerate(arr):
    if majority_consensus(scores[:-1], target_score=3, consensus_thresh=threshhold) or majority_consensus(scores, target_score=4, consensus_thresh=threshhold):
        consensus_slides.append(index)



In [ ]:
consensus_slides

Now check 3/3 consensus

In [ ]:
consensus_slides = []
threshhold = 3
for index, scores in enumerate(arr):
    if majority_consensus(scores[:-1], target_score=3, consensus_thresh=threshhold) or majority_consensus(scores, target_score=4, consensus_thresh=threshhold):
        consensus_slides.append(index)